In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
df = pd.read_csv("/content/gdrive/My Drive/vc_modeling/model_output/predictions.csv")
df.head

<bound method NDFrame.head of                          hash  ... prediction_2000
0        13685534557686295101  ...        1.579223
1          764015621929367586  ...             NaN
2        10846552445983457719  ...        0.874910
3        10693046220981818130  ...             NaN
4         5087506707876194815  ...        4.063588
...                       ...  ...             ...
1089728   7648380604111671063  ...             NaN
1089729  15062640940509416191  ...             NaN
1089730   9929788675484222851  ...             NaN
1089731   2009917061565156091  ...             NaN
1089732  12539503581336717831  ...             NaN

[1089733 rows x 16 columns]>

In [4]:
df.columns

Index(['hash', 'uuid', 'name', 'created_at', 'initial_valuation', 'truth_200',
       'prediction_200', 'initial_valuation500', 'truth_500', 'prediction_500',
       'initial_valuation1000', 'truth_1000', 'prediction_1000',
       'initial_valuation2000', 'truth_2000', 'prediction_2000'],
      dtype='object')

In [5]:
df = pd.DataFrame(df[["uuid", "initial_valuation", "prediction_200", "prediction_500", "prediction_1000", "prediction_2000"]])

In [6]:
df

,uuid,initial_valuation,prediction_200,prediction_500,prediction_1000,prediction_2000
0,e1393508-30ea-8a36-3f96-dd3226033abd,26250000.0,0.568490,1.017972,1.380360,1.579223
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,NaN,NaN,NaN,NaN,NaN
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,14000000.0,0.266237,0.493032,0.614898,0.874910
3,f4d5ab44-058b-298b-ea81-380e6e9a8eec,NaN,NaN,NaN,NaN,NaN
4,df662812-7f97-0b43-9d3e-12f64f504fbb,2500000.0,0.727375,1.646667,2.819469,4.063588
...,...,...,...,...,...,...
1089728,563edf19-174e-410c-a33b-85df066f4bdb,NaN,NaN,NaN,NaN,NaN
1089729,e690cf82-eda4-4065-a42f-1119708aae45,NaN,NaN,NaN,NaN,NaN
1089730,d206b3b0-c576-4557-aee5-1fb0e11b0afe,NaN,NaN,NaN,NaN,NaN
1089731,bdd81efe-0faa-43ac-99ab-0d964c160be1,NaN,NaN,NaN,NaN,NaN


In [7]:
df.dropna()

,uuid,initial_valuation,prediction_200,prediction_500,prediction_1000,prediction_2000
0,e1393508-30ea-8a36-3f96-dd3226033abd,26250000.0,0.568490,1.017972,1.380360,1.579223
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,14000000.0,0.266237,0.493032,0.614898,0.874910
4,df662812-7f97-0b43-9d3e-12f64f504fbb,2500000.0,0.727375,1.646667,2.819469,4.063588
8,60485007-8856-bbac-aa1b-c535c41f5f47,4000000.0,0.463736,1.011209,1.392781,1.526805
9,4111dc8b-c0df-2d24-ed33-30cd137b3098,10000000.0,0.755307,1.429051,1.746479,1.885427
...,...,...,...,...,...,...
1089708,784fb1d8-99f5-45e8-9cf7-bd95a4cc0314,18625040.0,0.244025,0.446312,0.607317,0.738452
1089713,408d924d-1c6f-48f8-8faa-0d152ed48ffa,11650010.0,0.240382,0.424526,0.585904,0.662069
1089717,ee597ac9-cb2d-4f63-8587-2f2d77b05a11,8049985.0,0.692821,1.378121,1.478332,1.735418
1089722,b6f1e202-645f-462f-a165-7459dd59d9b4,2875000.0,0.256831,0.467427,0.642284,0.726505


In [8]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from copy import deepcopy

In [9]:
file_names = ["organizations", "jobs", "people"]


In [10]:
dfs = [pd.read_csv("/content/gdrive/My Drive/vc_modeling/data/crunchbase_bulk_export/"+x+".csv") for x in file_names]
df_dict = dict(zip(file_names, dfs))
print(df_dict.keys())

dict_keys(['organizations', 'jobs', 'people'])


In [11]:
jobs_df = df_dict["jobs"]
jobs_founders = jobs_df[jobs_df.apply(lambda row: row["title"].lower().find("found") > -1 if isinstance(row["title"], str) else False, axis=1)]
org_and_founder_uuid = jobs_founders[["org_uuid", "person_uuid"]]
founders_uuids = jobs_founders.person_uuid.unique()

people_df = df_dict["people"]
org_founders = people_df.loc[people_df["uuid"].isin(founders_uuids)].set_index("uuid")
org_founders.head()

,name,type,permalink,cb_url,rank,created_at,updated_at,first_name,last_name,gender,country_code,state_code,region,city,featured_job_organization_uuid,featured_job_organization_name,featured_job_title,facebook_url,linkedin_url,twitter_url,logo_url
uuid,,,,,,,,,,,,,,,,,,,,,
ed13cd36-fe2b-3707-197b-0c2d56e37a71,Ben Elowitz,person,ben-elowitz,https://www.crunchbase.com/person/ben-elowitz,20738.0,2007-05-25 14:03:54,2020-02-21 05:19:55,Ben,Elowitz,male,USA,WA,Washington,Seattle,cf253887-5eac-21a2-28d3-47db7311f7e9,Madrona Venture Group,Managing Director,http://www.facebook.com/elowitz,https://www.linkedin.com/in/elowitz/,http://twitter.com/elowitz,https://crunchbase-production-res.cloudinary.c...
80d25c23-9726-9dda-5852-39cdf4810ea5,Ron Gorodetzky,person,ron-gorodetzky,https://www.crunchbase.com/person/ron-gorodetzky,91964.0,2007-05-26 03:07:50,2018-02-12 23:42:44,Ron,Gorodetzky,male,NaN,NaN,NaN,NaN,aa3bf156-06af-5b6e-215d-9e7211fc173b,fflick,Co-Founder and CTO,NaN,https://twitter.com/ronwinbeta,NaN,https://crunchbase-production-res.cloudinary.c...
a01b8d46-d311-3333-7c34-aa3ae9c03f22,Mark Zuckerberg,person,mark-zuckerberg,https://www.crunchbase.com/person/mark-zuckerberg,59.0,2007-05-26 04:51:46,2018-12-13 14:05:34,Mark,Zuckerberg,male,USA,CA,California,Palo Alto,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,Founder & CEO,https://www.facebook.com/zuck,NaN,https://twitter.com/finkd,https://crunchbase-production-res.cloudinary.c...
084aaa07-0795-1fe8-9c46-98bbeb02cd64,Dustin Moskovitz,person,dustin-moskovitz,https://www.crunchbase.com/person/dustin-mosko...,506.0,2007-05-26 05:15:05,2019-06-07 20:29:51,Dustin,Moskovitz,male,USA,CA,California,San Francisco,c2d47d05-9e78-de77-084c-dd5171251ff1,Asana,Co-Founder,http://www.facebook.com/moskov,https://www.linkedin.com/in/dmoskov,http://twitter.com/moskov,https://crunchbase-production-res.cloudinary.c...
d3ae96f0-4442-720f-1ec2-fc28eac94e53,Owen Van Natta,person,owen-van-natta,https://www.crunchbase.com/person/owen-van-natta,2161.0,2007-05-26 05:15:59,2019-08-30 06:14:27,Owen,Van Natta,male,USA,CA,California,San Francisco,e486ad63-e064-ce03-b644-a2ecd9dcc355,415,Founder and Managing Partner,NaN,https://www.linkedin.com/in/owen-van-natta-444a7/,NaN,https://crunchbase-production-res.cloudinary.c...


In [12]:
org_founders
org_founders.nunique()

name                              250697
type                                   1
permalink                         264839
cb_url                            264839
rank                              263977
created_at                        243126
updated_at                        121959
first_name                         39054
last_name                         119142
gender                                14
country_code                         169
state_code                            64
region                              1136
city                                6134
featured_job_organization_uuid    142198
featured_job_organization_name    140582
featured_job_title                 27572
facebook_url                       39330
linkedin_url                      180748
twitter_url                        71859
logo_url                          215561
dtype: int64

In [13]:
df_dict["organizations"].head()

,uuid,name,type,permalink,cb_url,rank,created_at,updated_at,legal_name,roles,domain,homepage_url,country_code,state_code,region,city,address,postal_code,status,short_description,category_list,category_groups_list,num_funding_rounds,total_funding_usd,total_funding,total_funding_currency_code,founded_on,last_funding_on,closed_on,employee_count,email,phone,facebook_url,linkedin_url,twitter_url,logo_url,alias1,alias2,alias3,primary_role,num_exits
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,organization,wetpaint,https://www.crunchbase.com/organization/wetpaint,123607.0,2007-05-25 13:51:27,2019-06-24 22:19:25,NaN,company,wetpaint.com,http://www.wetpaint.com/,USA,NY,New York,New York,902 Broadway 11th Floor New,10010,acquired,Wetpaint offers an online social publishing pl...,"Publishing,Social Media,Social Media Management","Content and Publishing,Internet Services,Media...",3.0,3.975000e+07,3.975000e+07,USD,2005-06-01,2008-05-19,NaN,51-100,info@wetpaint.com,206-859-6300,https://www.facebook.com/Wetpaint,https://www.linkedin.com/company/wetpaint,https://twitter.com/wetpainttv,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,company,NaN
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,organization,zoho,https://www.crunchbase.com/organization/zoho,8777.0,2007-05-26 02:30:28,2018-10-27 00:29:49,NaN,"investor,company",zoho.com,https://www.zoho.com/,USA,CA,California,Pleasanton,4141 Hacienda Drive,94588,operating,"Zoho offers a suite of business, collaboration...","Cloud Computing,Collaboration,CRM,Developer To...","Information Technology,Internet Services,Priva...",NaN,NaN,NaN,NaN,1996-09-15,NaN,NaN,1001-5000,info@zohocorp.com,NaN,http://www.facebook.com/zoho,http://www.linkedin.com/company/zoho-corporati...,http://twitter.com/zoho,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,company,1.0
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,organization,digg,https://www.crunchbase.com/organization/digg,11537.0,2007-05-26 03:03:23,2018-12-10 10:09:14,"Digg Holdings, LLC",company,digg.com,http://www.digg.com,USA,NY,New York,New York,NaN,NaN,acquired,Digg Inc. operates a website that enables its ...,"Internet,Social Media,Social Network","Internet Services,Media and Entertainment",6.0,4.900000e+07,4.900000e+07,USD,2004-10-11,2016-09-13,NaN,51-100,feedback@digg.com,877-342-7222,http://www.facebook.com/digg,http://www.linkedin.com/company/digg,http://twitter.com/digg,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,company,NaN
3,f4d5ab44-058b-298b-ea81-380e6e9a8eec,Omidyar Network,organization,omidyar-network,https://www.crunchbase.com/organization/omidya...,6844.0,2007-05-26 03:21:34,2019-06-19 12:17:48,NaN,investor,omidyar.com,http://www.omidyar.com,USA,CA,California,Redwood City,1991 Broadway Suite 200,94063,operating,Omidyar Network is an investment firm.,"Enterprise Software,Financial Services,Venture...","Financial Services,Lending and Investments,Sof...",NaN,NaN,NaN,NaN,2004-01-01,NaN,NaN,101-250,info@omidyar.com,650.482.2500,http://www.facebook.com/OmidyarNetwork,http://www.linkedin.com/company/22806,http://twitter.com/OmidyarNetwork,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,investor,33.0
4,df662812-7f97-0b43-9d3e-12f64f504fbb,Facebook,organization,facebook,https://www.crunchbase.com/organization/facebook,15.0,2007-05-26 04:22:15,2020-03-30 18:33:35,"Facebook, Inc.","investor,company",facebook.com,http://www.facebook.com,USA,CA,California,Menlo Park,1 Hacker Way,94025,ipo,Facebook is an online social networking servic...,"E-Commerce,Mobile Apps,Social,Social Media,Soc...","Apps,Commerce and Shopping,Content and Publish...",15.0,2.335700e+09,2.335700e+09,USD,2004-02-04,2013-06-30,NaN,10000+,NaN,NaN,https://www.facebook.com/facebook/,http://www.linkedin.com/company/facebook,https://twitter.com/facebook,https://crunchbase-production-res.cloudinary.c...,NaN,NaN,NaN,company,NaN


In [14]:
big_df = pd.concat([df.set_index('uuid'),pd.DataFrame(df_dict["organizations"][["uuid","name","short_description","founded_on"]]).set_index('uuid')], axis=1, join='inner')

In [15]:
big_df

,initial_valuation,prediction_200,prediction_500,prediction_1000,prediction_2000,name,short_description,founded_on
uuid,,,,,,,,
e1393508-30ea-8a36-3f96-dd3226033abd,26250000.0,0.568490,1.017972,1.380360,1.579223,Wetpaint,Wetpaint offers an online social publishing pl...,2005-06-01
bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,NaN,NaN,NaN,NaN,NaN,Zoho,"Zoho offers a suite of business, collaboration...",1996-09-15
5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,14000000.0,0.266237,0.493032,0.614898,0.874910,Digg,Digg Inc. operates a website that enables its ...,2004-10-11
f4d5ab44-058b-298b-ea81-380e6e9a8eec,NaN,NaN,NaN,NaN,NaN,Omidyar Network,Omidyar Network is an investment firm.,2004-01-01
df662812-7f97-0b43-9d3e-12f64f504fbb,2500000.0,0.727375,1.646667,2.819469,4.063588,Facebook,Facebook is an online social networking servic...,2004-02-04
...,...,...,...,...,...,...,...,...
563edf19-174e-410c-a33b-85df066f4bdb,NaN,NaN,NaN,NaN,NaN,StockLyft,StockLyft is a CSA Shares and farm micro loans...,2020-02-19
e690cf82-eda4-4065-a42f-1119708aae45,NaN,NaN,NaN,NaN,NaN,The SEO Noob,Digital Marketing,2020-05-30
d206b3b0-c576-4557-aee5-1fb0e11b0afe,NaN,NaN,NaN,NaN,NaN,Tellaport,Virtual Teleportation Platform,NaN


In [16]:
big_df.dropna()

,initial_valuation,prediction_200,prediction_500,prediction_1000,prediction_2000,name,short_description,founded_on
uuid,,,,,,,,
e1393508-30ea-8a36-3f96-dd3226033abd,26250000.0,0.568490,1.017972,1.380360,1.579223,Wetpaint,Wetpaint offers an online social publishing pl...,2005-06-01
5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,14000000.0,0.266237,0.493032,0.614898,0.874910,Digg,Digg Inc. operates a website that enables its ...,2004-10-11
df662812-7f97-0b43-9d3e-12f64f504fbb,2500000.0,0.727375,1.646667,2.819469,4.063588,Facebook,Facebook is an online social networking servic...,2004-02-04
60485007-8856-bbac-aa1b-c535c41f5f47,4000000.0,0.463736,1.011209,1.392781,1.526805,Omnidrive,Omnidrive provides online storage facilities t...,2005-11-01
4111dc8b-c0df-2d24-ed33-30cd137b3098,10000000.0,0.755307,1.429051,1.746479,1.885427,Geni,Geni is an online community of genealogists co...,2006-06-01
...,...,...,...,...,...,...,...,...
784fb1d8-99f5-45e8-9cf7-bd95a4cc0314,18625040.0,0.244025,0.446312,0.607317,0.738452,CytoVeris,CytoVeris develops optical-based probes enabli...,2018-01-01
408d924d-1c6f-48f8-8faa-0d152ed48ffa,11650010.0,0.240382,0.424526,0.585904,0.662069,Elgia Therapeutics,Elgia Therapeutics specializes in novel approa...,2019-01-01
ee597ac9-cb2d-4f63-8587-2f2d77b05a11,8049985.0,0.692821,1.378121,1.478332,1.735418,Cloud Paper,Cloud Paper is committed in manufacturing pape...,2019-01-01


In [17]:
founders = org_founders.groupby("featured_job_organization_uuid")

In [18]:
founder_names = org_founders.groupby("featured_job_organization_uuid")['name'].apply(', '.join).reset_index()

In [19]:
founder_names.columns = ["featured_job_organization_uuid", "founder_names"]
founder_names

,featured_job_organization_uuid,founder_names
0,000014da-0c46-b9cb-0941-3a93c027b119,"Anders Søndergaard, Thomas Scherning, Helga Ha..."
1,00002470-bff7-6226-5800-0ca1b3787b6f,Weiting Liu
2,00004c41-aeeb-b5a0-8812-64cf2d017cfb,Richard Florida
3,00007c5c-9260-0dfb-c160-89a416f1a7cc,"Konrad Kopczynski, Alika Graham"
4,0000c0e1-eb00-9281-9a7f-63dfa277e8df,Seffie Treistman
...,...,...
142193,fffe2508-e9e3-8602-9664-8c6ef2c803de,Susan Burke
142194,fffeb0c0-db98-4585-a40d-d36b140ae446,"Valentina Shegoyan, Roman Slivin"
142195,ffff2c3b-ca66-67dc-c643-77cf3a5f89e2,Edgar Moreno
142196,ffff8343-97a6-3667-67af-ed3789fa9e14,Brittany Reiff


In [20]:
big_df2 = pd.concat([big_df,founder_names.set_index("featured_job_organization_uuid")], axis=1, join='inner')

In [21]:
big_df2 = big_df2.dropna()

In [22]:
big_df2

,initial_valuation,prediction_200,prediction_500,prediction_1000,prediction_2000,name,short_description,founded_on,founder_names
df662812-7f97-0b43-9d3e-12f64f504fbb,2500000.0,0.727375,1.646667,2.819469,4.063588,Facebook,Facebook is an online social networking servic...,2004-02-04,"Mark Zuckerberg, Blake Williams, Ashwin Bharam..."
5da6106f-0d27-0d37-e9d7-dcfeccc1f709,25000000.0,0.875369,1.667170,2.658840,4.160244,Twitter,Twitter is a social networking platform that a...,2006-03-21,"Jack Dorsey, Biz Stone, John Kalucki, Britt Se..."
3d16cb4c-911e-75c0-de5a-15c316b39f98,7500000.0,0.697123,1.420054,1.492200,1.747966,StumbleUpon,StumbleUpon is a discovery engine that finds a...,2001-01-01,"Geoff Smith, Eric Boyd"
21e77067-5537-408e-cad7-e5e72bb6ad86,200000.0,0.674236,1.154655,1.525770,1.775467,Scribd,Scribd is an unlimited reading subscription th...,2007-03-01,"John R. Adler, Colton Gyulay"
e56b0ceb-bb30-bbec-805e-d5dc7412dcb1,33500000.0,0.698910,1.373838,1.521852,1.831529,eBay,eBay is an online marketplace that connects a ...,1995-09-01,"Pierre Omidyar, Brian McGarvey, Steve Yankovic..."
...,...,...,...,...,...,...,...,...,...
e682eef3-1a6b-4959-aba6-d05a28937c66,750000.0,0.273632,0.413797,0.570863,0.644364,Boost Profits,Boost Profits is an ISO that provides a disru...,2020-01-01,Rocky Emery
a346472b-fc10-4b62-a433-f4ce14064960,5250000.0,0.294365,0.563761,0.724946,0.777358,Disaster Tech,Disaster Tech is a data science tools and cust...,2019-01-01,Sean M. Griffin
d9b8c000-17f0-4048-b056-390078bdc01b,412490.0,0.287193,0.501320,0.694345,0.720208,Bopdrop,Bopdrop is an application that makes it easy t...,2019-01-01,Matthew Krause
ebc63a49-54c3-4222-bef2-ec559ca1e9cc,57875000.0,0.252980,0.446311,0.648885,0.764686,DxTx Pain and Spine,DxTx Pain and Spine is a health network of aff...,2019-01-01,Owen V. Prunskis


In [23]:
big_df2["prediction_200"] = big_df2["prediction_200"].apply(lambda x: (np.exp(x)-1) * 100)
big_df2["prediction_500"] = big_df2["prediction_500"].apply(lambda x: (np.exp(x)-1) * 100)
big_df2["prediction_1000"] = big_df2["prediction_1000"].apply(lambda x: (np.exp(x)-1) * 100)
big_df2["prediction_2000"] = big_df2["prediction_2000"].apply(lambda x: (np.exp(x)-1) * 100)

In [24]:
big_df2

,initial_valuation,prediction_200,prediction_500,prediction_1000,prediction_2000,name,short_description,founded_on,founder_names
df662812-7f97-0b43-9d3e-12f64f504fbb,2500000.0,106.964157,418.965388,1576.794758,5718.271677,Facebook,Facebook is an online social networking servic...,2004-02-04,"Mark Zuckerberg, Blake Williams, Ashwin Bharam..."
5da6106f-0d27-0d37-e9d7-dcfeccc1f709,25000000.0,139.976013,429.715495,1327.971950,6308.713645,Twitter,Twitter is a social networking platform that a...,2006-03-21,"Jack Dorsey, Biz Stone, John Kalucki, Britt Se..."
3d16cb4c-911e-75c0-de5a-15c316b39f98,7500000.0,100.796699,313.734352,344.686652,474.291078,StumbleUpon,StumbleUpon is a discovery engine that finds a...,2001-01-01,"Geoff Smith, Eric Boyd"
21e77067-5537-408e-cad7-e5e72bb6ad86,200000.0,96.253226,217.292758,359.868368,490.303627,Scribd,Scribd is an unlimited reading subscription th...,2007-03-01,"John R. Adler, Colton Gyulay"
e56b0ceb-bb30-bbec-805e-d5dc7412dcb1,33500000.0,101.155809,295.048298,358.069861,524.342823,eBay,eBay is an online marketplace that connects a ...,1995-09-01,"Pierre Omidyar, Brian McGarvey, Steve Yankovic..."
...,...,...,...,...,...,...,...,...,...
e682eef3-1a6b-4959-aba6-d05a28937c66,750000.0,31.473111,51.255047,76.979332,90.477516,Boost Profits,Boost Profits is an ISO that provides a disru...,2020-01-01,Rocky Emery
a346472b-fc10-4b62-a433-f4ce14064960,5250000.0,34.227409,75.726917,106.461925,117.571620,Disaster Tech,Disaster Tech is a data science tools and cust...,2019-01-01,Sean M. Griffin
d9b8c000-17f0-4048-b056-390078bdc01b,412490.0,33.268105,65.089897,100.239777,105.486150,Bopdrop,Bopdrop is an application that makes it easy t...,2019-01-01,Matthew Krause
ebc63a49-54c3-4222-bef2-ec559ca1e9cc,57875000.0,28.785757,56.253725,91.340710,114.832032,DxTx Pain and Spine,DxTx Pain and Spine is a health network of aff...,2019-01-01,Owen V. Prunskis


In [25]:
big_df2.to_csv("cleaned_outputs.csv")

In [26]:
len(big_df2)

56227

In [27]:
big_df2 = big_df2.dropna()

In [28]:
len(big_df2)

56227

In [30]:
big_df2['prediction_200'].mean()

46.468884246633074

In [31]:
big_df2['prediction_2000'].mean()

221.77438448321266